Zhiqin Yang - zy2425

In [15]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup


url = "https://www.thereformation.com/clothing?page=100" # the total pages won't be more than 100, so set page = 100 includes all the products
response = requests.get(url)
results_page = BeautifulSoup(response.content,'lxml') 
home = "https://www.thereformation.com/"

In [16]:
wholepage = results_page.find_all('div', {'class':"product-tile product-tile--default"}) #contains a brief introduction of the products

In [17]:
name = []
price = []
category = []
color = []
surl = []
img = []
for i in wholepage:
    item = i.get("data-aggregate")
    if item == '{"trackObject": {"ecommerce":{"click":{"products":[{"id":"1303674","name":"Patagonia Men':
        break # because we set page = 100 which is too big, so we stop when the contents includes an error. 
    item = eval(item)
    name.append(item['trackObject']['ecommerce']['click']['products'][0]['name'])
    price.append(item['trackObject']['ecommerce']['click']['products'][0]['price'])
    category.append(item['trackObject']['ecommerce']['click']['products'][0]['category'])
    color.append(item['trackObject']['ecommerce']['click']['products'][0]['dimension1'])#color
    href = home+i.find('a',{'class':"product-tile__anchor"}).get('href')#link
    surl.append(href)
    img.append(i.find('img').get('data-src'))#img

In [ ]:
size = []
for i in surl: 
    suburl = i
    response = requests.get(suburl)
    sub = BeautifulSoup(response.content,'lxml') 
    search = sub.find_all('div',{'class':"product-attribute__contents flex flex-flow-wrap"})
    if len(search) > 1:
        sizes = search[1].get_text()[1:-1].replace('\n',',')#[1:-1] -> we delete the unexpected '\n'
    else:
        sizes = " "
    sizes = sizes.split(",")
    size.append(sizes)

In [ ]:
des = []
for i in surl: 
    suburl = i
    response = requests.get(suburl)
    sub = BeautifulSoup(response.content,'lxml') 
    search = sub.find_all('div',{'class':"cms-generic-copy"})
    if len(search) > 1:
        subdes = search[1].get_text()[1:-1]
    else:
        subdes = ' '
    des.append(subdes)

In [ ]:
material = []
for i in surl: 
    suburl = i
    response = requests.get(suburl)
    sub = BeautifulSoup(response.content,'lxml') 
    search = sub.find_all('div',{'class':"margin-b--15"})
    if len(search) > 3:
        fabric = search[3].get_text()[1:-1]
    else:
        fabric = ' '
    material.append(fabric)

In [ ]:
import pandas as pd
scrap_result = pd.DataFrame(name,columns = ["Product Name"])

In [ ]:
scrap_result["Price"] = price
scrap_result["Category"] = category
scrap_result["Color"] = color
scrap_result["Size"] = size
scrap_result["Material"] = material
scrap_result["Description"] = des
scrap_result["Link"] = surl
scrap_result["Img Link"] = img #form the table we want

In [ ]:
from datetime import datetime
now = datetime.now().time()
timenow = [now.strftime("%Y-%m-%d %H:%M:%S")]
scrap_result["Scrapped Date"] = timenow * len(name)# show the date we scrap these information
# because reformation only sells clothing for women, so I didn't include the "gender" and "secondhand" in the table

In [35]:
!pip install psycopg2

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://postgres@localhost:5432/Reformation')
scrap_result.to_sql('scrap_result',engine,index=False,if_exists='append')
# the codes we need to update our data into AWS RDS is similar